In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# ResNet

![对于非嵌套函数类，较复杂（由较大区域表示）的函数类不能保证更接近“真”函数（ $f^*$ ）。这种现象在嵌套函数类中不会发生。](http://d2l.ai/_images/functionclasses.svg)


## 残差块
![avater](https://zh-v2.d2l.ai/_images/residual-block.svg)
上图是一个正常块（左图）和一个残差块（右图）左图虚线框中的部分需要直接拟合出该映射  𝑓(𝐱)  ，而右图虚线框中的部分则需要拟合出残差映射  𝑓(𝐱)−𝐱  。 残差映射在现实中往往更容易优化。右图是 ResNet 的基础结构– 残差块（residual block）。 在残差块中，输入可通过跨层数据线路更快地向前传播。

ResNet 沿用了 VGG 完整的  3×3  卷积层设计。 残差块里首先有 2 个有相同输出通道数的  3×3  卷积层。 每个卷积层后接一个批量归一化层和 ReLU 激活函数。 然后我们通过跨层数据通路，跳过这 2 个卷积运算，将输入直接加在最后的 ReLU 激活函数前。 这样的设计要求 2 个卷积层的输出与输入形状一样，从而可以相加。 如果想改变通道数，就需要引入一个额外的  1×1  卷积层来将输入变换成需要的形状后再做相加运算。 残差块的实现如下：

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


class Residual(nn.Module):  
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

此代码生成两种类型的网络： 一种是在 use_1x1conv=False 、应用 ReLU 非线性函数之前，将输入添加到输出。 另一种是在 use_1x1conv=True 时，添加通过  1×1  卷积调整通道和分辨率。
![avater](https://zh-v2.d2l.ai/_images/resnet-block.svg)


下面我们来查看输入和输出形状一致的情况

In [3]:
blk = Residual(3,3)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
Y.shape

torch.Size([4, 3, 6, 6])

我们也可以在增加输出通道数的同时，减半输出的高和宽。

In [4]:
blk = Residual(3,6, use_1x1conv=True, strides=2)
blk(X).shape

torch.Size([4, 6, 3, 3])

## ResNet模型

ResNet 的前两层跟之前介绍的 GoogLeNet 中的一样： 在输出通道数为 64、步幅为 2 的  7×7  卷积层后，接步幅为 2 的  3×3  的最大汇聚层。 不同之处在于 ResNet 每个卷积层后增加了批量归一化层。

In [5]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [6]:
def resnet_block(input_channels, num_channels, num_residuals,first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

我们对第一个模块做了特别处理。接着在 ResNet 加入所有残差块，这里每个模块使用 2 个残差块。

In [7]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

最后，与 GoogLeNet 一样，在 ResNet 中加入全局平均汇聚层，以及全连接层输出

In [8]:
net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), 
                    nn.Linear(512, 10))

每个模块有 4 个卷积层（不包括恒等映射的 1×1卷积层，具体为每个resnet_block包含2个Residual,1个Residual中有2次卷积）。 加上第一个  7×7  卷积层和最后一个全连接层，共有 18 层。 因此，这种模型通常被称为 ResNet-18。
![avater](https://zh-v2.d2l.ai/_images/resnet18.svg)

In [9]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


In [10]:
#训练
lr, num_epochs, batch_size = 0.05, 10, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

Mac由于没有GPU，这里通过markdown直接展示结果。

![avatar](https://zh-v2.d2l.ai/_images/output_resnet_46beba_102_1.svg)

loss 0.014, &ensp; train acc 0.996,&ensp; test acc 0.895

4680.2 examples/sec on cuda:0

## 小结

* 学习嵌套函数（nested function）是训练神经网络的理想情况。在深层神经网络中，学习另一层作为恒等映射（identity function）较容易（尽管这是一个极端情况）。
* 残差映射可以更容易地学习同一函数，例如将权重层中的参数近似为零。
* 利用残差块（residual blocks）可以训练出一个有效的深层神经网络：输入可以通过层间的残余连接更快地向前传播。
* 残差网络（ResNet）对随后的深层神经网络设计产生了深远影响，无论是卷积类网络还是全连接类网络。